In [10]:
import pandas as pd
import glob
import os
import re

In [11]:
data = pd.read_pickle('/Users/gocchini/Desktop/paper_3/data/data_almost_complete.pkl')
london_postcodes = pd.read_csv('/Users/gocchini/Desktop/paper_3/data/london_postcodes-ons-postcodes-directory-feb22.csv')

In [12]:
def data_processing(data, london_data): 
    data = data.loc[(data['Inactive'] == 'No')]
    data = data[data['Primary UK SIC (2007) code'].notna()]
    data = data[data['Primary trading address'].notna()]
    data = data.drop_duplicates(keep=False)
    data['postcode'] = data['Primary trading address'].apply(lambda x: x.split(',')[-1])
    data['postcode'] = data['postcode'].apply(lambda x: x[1:])
    london_postcodes = london_data['pcd'].tolist()
    london_df = data[data['postcode'].isin(london_postcodes)]
    london_df['Primary UK SIC (2007) code'] = london_df['Primary UK SIC (2007) code'].astype('string')
    return london_df 

In [13]:

london_df = data_processing(data , london_postcodes)

/var/folders/0f/pzhfvnbj63jg3sk3nj099q3w0000gq/T/ipykernel_19333/2154885762.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  london_df['Primary UK SIC (2007) code'] = london_df['Primary UK SIC (2007) code'].astype('string')


In [14]:
london_intangibles = london_df[london_df['Primary UK SIC (2007) code'].str.contains(
    "^85|^92|^96|^59|^60|^61|^62|^63|^64|^66|^69|^70|^71|^72|^73|^74|^79|^82|^77|^78")]

In [17]:
text3 = pd.read_pickle('/Users/gocchini/Desktop/paper_1/scripts_paper_1/df_november_2022.pkl')
text3['actual_url'] = text3['url'].str.replace('http://', '')
text3['actual_url'] = text3['actual_url'].str.replace('https://', '')

In [19]:
london_data = london_intangibles.merge(text3, left_on = 'Website', right_on= 'actual_url')

In [20]:
london_data = london_data.loc[london_data['status']==True]

In [54]:
england_postcodes = pd.read_csv('/Users/gocchini/Downloads/England postcodes.csv')

/var/folders/0f/pzhfvnbj63jg3sk3nj099q3w0000gq/T/ipykernel_19333/2786139566.py:1: DtypeWarning: Columns (18,29,47) have mixed types. Specify dtype option on import or set low_memory=False.
  england_postcodes = pd.read_csv('/Users/gocchini/Downloads/England postcodes.csv')


In [56]:
english_postcodes = england_postcodes['Postcode'].tolist()

In [27]:
def postcode_finder(text):
    postcodes = re.findall(r'([Gg][Ii][Rr] 0[Aa]{2})|((([A-Za-z][0-9]{1,2})|(([A-Za-z][A-Ha-hJ-Yj-y][0-9]{1,2})|(([A-Za-z][0-9][A-Za-z])|([A-Za-z][A-Ha-hJ-Yj-y][0-9][A-Za-z]?))))\s?[0-9][A-Za-z]{2})', text)
    postcodes = [postcode[1] for postcode in postcodes]
    return postcodes 

In [28]:
london_data['text_postcode'] = london_data['text'].apply(postcode_finder)

In [59]:
def suspicious_postcodes_finder(postcode, postcode_in_text, english_postcodes):
    if len(postcode_in_text) > 0: 
        if postcode_in_text[0].isupper():
            if postcode_in_text[0] in english_postcodes:
                if postcode != postcode_in_text[0]:
                    return 'SUSPICIOUS!'
                else: 
                    return 'ok'

In [60]:
london_data['suspicious'] = london_data.apply(lambda x: suspicious_postcodes_finder(x['postcode'], x['text_postcode'], english_postcodes), axis = 1)

In [61]:
london_data

,Unnamed: 0,Company name,Inactive,Quoted,Branch,OwnData,Woco,R/O Full Postcode,Primary UK SIC (2007) code,Primary trading address,...,"R/O Address, line 3",R/O Full Postcode.1,Primary trading address City,postcode,url,status,text,actual_url,text_postcode,suspicious
0,84.0,MOORE STEPHENS INTERNATIONAL LIMITED,No,No,No,No,No,SE1 3TQ,96090.0,"5 Market Yard Mews, 194-204 Bermondsey Street,...",...,NaN,SE1 3TQ,London,SE1 3TQ,http://www.sequel.com,True,\n\n\n\n\n\n\nVerisk Sequel - The future of in...,www.sequel.com,[],None
1,320.0,PULSTAR DATAPRINT LIMITED,No,No,No,No,No,NW2 7HD,62090.0,"152-154 Coles Green Road, London, London, NW2 7HD",...,NaN,NW2 7HD,London,NW2 7HD,http://www.pulstar.co.uk,True,\n\n\nHome :: Osmosis Promotional Products \n ...,www.pulstar.co.uk,[EN5 4DJ],SUSPICIOUS!
2,1292.0,SEGA EUROPE OVERSEAS LIMITED,No,No,No,No,No,TW8 9BW,70100.0,"27 Great West Road, Brentford, Middlesex, TW8 9BW",...,NaN,TW8 9BW,Brentford,TW8 9BW,http://www.sega.com,True,\n\nWeb Page Blocked\n\n\n\n\n\n\n\nWeb Page B...,www.sega.com,[],None
3,5513.0,SEGA EUROPE LIMITED,No,No,No,No,No,TW8 9BW,96090.0,"27 Great West Road, Brentford, Middlesex, TW8 9BW",...,NaN,TW8 9BW,Brentford,TW8 9BW,http://www.sega.com,True,\n\nWeb Page Blocked\n\n\n\n\n\n\n\nWeb Page B...,www.sega.com,[],None
4,53559.0,TWO POINT STUDIOS LIMITED,No,No,No,No,No,TW8 9BW,62011.0,"C/0 Sega Europe Limited, 27, Great West Road, ...",...,NaN,TW8 9BW,Brentford,TW8 9BW,http://www.sega.com,True,\n\nWeb Page Blocked\n\n\n\n\n\n\n\nWeb Page B...,www.sega.com,[],None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7934,72757.0,SRI VITTAL INFOTECH LIMITED,No,No,No,No,No,SM6 9LZ,62020.0,"15 Mallard Way, Wallington, Surrey, SM6 9LZ",...,NaN,SM6 9LZ,Wallington,SM6 9LZ,http://technologyonice.blogspot.com,True,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nTechnology on Ic...,technologyonice.blogspot.com,[],None
7935,72843.0,EDGWARE DRYCLEANERS LTD,No,No,No,No,No,HA8 7LY,96010.0,"43 Manor Park Crescent, Edgware, Middlesex, HA...",...,NaN,HA8 7LY,Edgware,HA8 7LY,http://www.ueni.com,True,UENI - The #1 Solution for Small Business Webs...,www.ueni.com,"[ss02Ou, te03Yo, ek04Ho]",None
7937,73092.0,INFINITY CONSULTING (UK) LIMITED,No,No,No,No,No,HA7 2DX,62012.0,"3 Bush Grove, Stanmore, Middlesex, HA7 2DX",...,NaN,HA7 2DX,Stanmore,HA7 2DX,http://www.infinitylimited.co.uk,True,,www.infinitylimited.co.uk,[],None
7938,73715.0,ANA SOLUTIONS LIMITED,No,No,No,No,No,IG1 3QH,62020.0,"58 Cranbrook Rise, Ilford, Essex, IG1 3QH",...,NaN,IG1 3QH,Ilford,IG1 3QH,http://www.anatriosolutions.com,True,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,www.anatriosolutions.com,[],None


In [62]:
london_data['suspicious'].tolist()

[None,
 'SUSPICIOUS!',
 None,
 None,
 None,
 None,
 'ok',
 None,
 'ok',
 'ok',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'SUSPICIOUS!',
 None,
 None,
 'ok',
 'SUSPICIOUS!',
 None,
 None,
 None,
 None,
 None,
 'SUSPICIOUS!',
 None,
 None,
 None,
 'ok',
 'ok',
 None,
 None,
 None,
 None,
 None,
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 'ok',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'ok',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'SUSPICIOUS!',
 'SUSPICIOUS!',
 'ok',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'ok',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'ok',
 None,
 None,
 'ok',
 None,
 'ok',
 'ok',
 'ok',
 None,
 'SUSPICIOUS!',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'ok',
 None,
 None,